In [1]:
import datasets 
import numpy as np 
from transformers import BertTokenizerFast 
from transformers import DataCollatorForTokenClassification 
from transformers import AutoModelForTokenClassification 
from sklearn.model_selection import train_test_split
from datasets import Dataset

import warnings
warnings.filterwarnings('ignore')

# https://github.com/rohan-paul/MachineLearning-DeepLearning-Code-for-my-YouTube-Channel/blob/master/NLP/YT_Fine_tuning_BERT_NER_v1.ipynb

/Users/julianbehrendt/opt/anaconda3/envs/computational_semantics/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-02 18:28:20.242189: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
full_sen = []
with open('train.en.txt') as fh:
    # Skip initial comments that starts with #
    while True:
        line = fh.readline()
        # break while statement if it is not a comment line
        # i.e. does not startwith #
        if not line.startswith('#'):
            full_sen.append(line) 
        if not line:
            break    


train_tags_all = []
train_token_all =[]

tokens = []
tags = []

train_data_en = []
train =[]
for i in range (0, len(full_sen)):
    string = full_sen[i].split("\t")
    if not len(full_sen[i]) == 0: 
        if string[0] == '\n':
            train_token_all.append(tokens) 
            tokens = []
            train_tags_all.append(tags)
            tags = []
            train_data_en.append(train)
            train = []
        else:
            tokens.append(string[0])
            tags.append(string[3])
            train.append((string[0],string[3]))

print("The English training data set has",len(train_data_en), "sentences." )

full_sen = []
with open('train.de.txt') as fh:
    # Skip initial comments that starts with #
    while True:
        line = fh.readline()
        # break while statement if it is not a comment line
        # i.e. does not startwith #
        if not line.startswith('#'):
            full_sen.append(line) 
        if not line:
            break    

tokens = []
tags = []
train_tags = []
train_token =[]
train_data_de = []
train =[]
for i in range (0, len(full_sen)):
    string = full_sen[i].split("\t")
    if not len(full_sen[i]) == 0: 
        if string[0] == '\n':
            train_token_all.append(tokens) 
            tokens = []
            train_tags_all.append(tags)
            tags = []
            train_data_de.append(train)
            train = []
        else:
            tokens.append(string[0])
            tags.append(string[3])
            train.append((string[0],string[3]))
            
print("The German training data set has",len(train_data_de), "sentences." )

full_sen = []
with open('train.nl.txt') as fh:
    # Skip initial comments that starts with #
    while True:
        line = fh.readline()
        # break while statement if it is not a comment line
        # i.e. does not startwith #
        if not line.startswith('#'):
            full_sen.append(line) 
        if not line:
            break    

tokens = []
tags = []
train_tags = []
train_token =[]
train_data_nl = []
train =[]
for i in range (0, len(full_sen)):
    string = full_sen[i].split("\t")
    if not len(full_sen[i]) == 0: 
        if string[0] == '\n':
            train_token_all.append(tokens) 
            tokens = []
            train_tags_all.append(tags)
            tags = []
            train_data_nl.append(train)
            train = []
        else:
            tokens.append(string[0])
            tags.append(string[3])
            train.append((string[0],string[3]))
            
print("The Dutch training data set has",len(train_data_nl), "sentences." )

full_sen = []
with open('train.it.txt') as fh:
    # Skip initial comments that starts with #
    while True:
        line = fh.readline()
        # break while statement if it is not a comment line
        # i.e. does not startwith #
        if not line.startswith('#'):
            full_sen.append(line) 
        if not line:
            break    

tokens = []
tags = []
train_tags = []
train_token =[]
train_data_it = []
train =[]
for i in range (0, len(full_sen)):
    string = full_sen[i].split("\t")
    if not len(full_sen[i]) == 0: 
        if string[0] == '\n':
            train_token_all.append(tokens) 
            tokens = []
            train_tags_all.append(tags)
            tags = []
            train_data_it.append(train)
            train = []
        else:
            tokens.append(string[0])
            tags.append(string[3])
            train.append((string[0],string[3]))
            
print("The Italian training data set has",len(train_data_it), "sentences." )

The English training data set has 7745 sentences.
The German training data set has 1740 sentences.
The Dutch training data set has 538 sentences.
The Italian training data set has 684 sentences.


In [3]:
print("all data has x tokens",len(train_token_all), "sentences." )
print("all data has x tokens",len(train_tags_all), "sentences." )

all data has x tokens 10707 sentences.
all data has x tokens 10707 sentences.


In [4]:
full_sen = []
with open('test.nl.txt') as fh:
    # Skip initial comments that starts with #
    while True:
        line = fh.readline()
        # break while statement if it is not a comment line
        # i.e. does not startwith #
        if not line.startswith('#'):
            full_sen.append(line) 
        if not line:
            break   
tokens = []
tags = []
test_token = []
test_tags =[]
test_data = []
test =[]
for i in range (0, len(full_sen)):
    string = full_sen[i].split("\t")
    if not len(full_sen[i]) == 0: 
        if string[0] == '\n':
            test_token.append(tokens) 
            tokens = []
            test_tags.append(tags)
            tags = []
            test_data.append(train)
            test = []
        else:
            tokens.append(string[0])
            tags.append(string[3])
            test.append((string[0],string[3]))

print("The testing data set has",len(test_data), "sentences.")

The testing data set has 490 sentences.


In [5]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased") 

In [6]:
mydict = np.load('universal_dict.npy',allow_pickle='TRUE').item()

In [7]:
def transform_into_ints(data,mydict):
    for sentences in range(0,len(data)):
        sent = data[sentences]
        for i in range(0,len(sent)):
            word = sent[i]
            transformation = mydict[word]
            sent[i] = transformation
    return data

In [8]:
train_tags_transformed = train_tags_all
test_tags_transformed = test_tags

transformed_input_train = transform_into_ints(train_tags_transformed,mydict)
transformed_input_test = transform_into_ints(test_tags_transformed,mydict)

In [9]:
X_train, X_val, y_train, y_val = train_test_split(
         train_token_all, transformed_input_train, test_size=0.3, random_state=18)

In [10]:
# check that sklearn shuffles the data. 
for i in range(0,20):
    print(X_train[i])

['Tom', 'wachte', 'um', 'halb~sieben', 'auf', '.']
['The', 'poor', 'cat', 'was', 'run', 'over', 'by', 'a', 'truck', '.']
['Jill', 'ist', 'mit', 'Jack', 'verlobt', '.']
['Napoleon~Bonaparte', 'was', 'born', 'in', 'Corsica', '.']
['You', 'tortured', 'a', 'prisoner', '.']
['Tom', 'is', 'now', 'apologizing', '.']
['I', 'love', 'dogs', '.']
['There', 'is', 'an', 'old', 'house', 'on', 'this', 'street', '.']
['This', 'desk', 'was', 'too', 'heavy', 'to', 'lift', '.']
['"', 'He', 'is', 'an', 'American', 'political~scientist', '.', '"']
['My', 'name', 'is', 'Tom', 'and', 'I', "'m", 'an', 'addict', '.']
['Kate', 'hat', 'das', 'Zimmer', 'nicht', 'gereinigt', '.']
['Tom', 'is', 'very', 'young', '.']
['I', 'refreshed', 'myself', 'with', 'a', 'hot', 'bath', '.']
['He', "'s", 'always', 'breaking', 'into', 'our', 'conversation', '.']
['She', "'s", 'taking', 'birth~control~pills', '.']
['She', "'s", 'painting', 'her', 'room', 'white', '.']
['Tom', 'did', "n't", 'like', 'his', 'sandwich', '.']
['Né', 'An

In [11]:
from collections import Counter

D = y_train

# Flatten the nested list
flattened_list = [item for sublist in D for item in sublist]

# Count the frequencies of each value
counter = Counter(flattened_list)

# Print the frequencies of each value
print(counter)

Counter({13: 7477, 1: 6050, 9: 3895, 21: 3314, 23: 2526, 18: 2520, 2: 2417, 5: 2325, 6: 1794, 12: 1647, 0: 1492, 4: 1233, 11: 1094, 34: 979, 7: 888, 20: 821, 19: 689, 26: 612, 3: 610, 31: 504, 24: 420, 29: 361, 14: 308, 39: 305, 16: 225, 45: 188, 44: 170, 15: 169, 49: 164, 38: 158, 43: 141, 25: 138, 37: 127, 17: 127, 53: 120, 41: 112, 22: 112, 8: 98, 10: 96, 42: 91, 33: 80, 51: 76, 50: 73, 35: 69, 28: 66, 52: 50, 27: 46, 30: 45, 47: 43, 40: 43, 46: 38, 62: 38, 58: 35, 36: 33, 32: 33, 48: 31, 55: 22, 61: 19, 54: 18, 63: 15, 59: 13, 57: 11, 67: 11, 60: 10, 64: 3, 65: 1, 68: 1})


In [12]:
def get_ids(tokens, tags):
    ids = []
    token = []
    ner_tags = []
    for i in range(0, len(tokens)):
        ids.append(i)
        token.append(tokens[i])
        ner_tags.append(tags[i])
    return ids, token, ner_tags

In [13]:
# To get it into the correct form: https://huggingface.co/docs/datasets/v1.1.1/loading_datasets.html

ids_train, tokens_train, ner_tags_train = get_ids(X_train, y_train)
ids_val, tokens_val, ner_tags_val = get_ids(X_val, y_val)

ids_test, tokens_test, ner_tags_test = get_ids(test_token, transformed_input_test)

train = {'input_ids': ids_train,
            'tokens': tokens_train,
            'ner_tags': ner_tags_train}

validation = {'input_ids': ids_val,
            'tokens': tokens_val,
            'ner_tags': ner_tags_val}

test = {'input_ids': ids_test,
            'tokens': tokens_test,
            'ner_tags': ner_tags_test}

train = Dataset.from_dict(train) 
validation = Dataset.from_dict(validation)
test = Dataset.from_dict(test)

data = {'train': train,
            'validation': validation,
            'test': test}

In [14]:
conll2003 = datasets.load_dataset("conll2003") 
conll2003

conll2003['train'] = data['train']
conll2003['validation'] = data['validation']
conll2003['test'] = data['test']

Found cached dataset conll2003 (/Users/julianbehrendt/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)
100%|██████████| 3/3 [00:00<00:00, 134.78it/s]


In [15]:
conll2003

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'tokens', 'ner_tags'],
        num_rows: 7494
    })
    validation: Dataset({
        features: ['input_ids', 'tokens', 'ner_tags'],
        num_rows: 3213
    })
    test: Dataset({
        features: ['input_ids', 'tokens', 'ner_tags'],
        num_rows: 490
    })
})

In [16]:
def tokenize_and_align_labels(examples, label_all_tokens=True): 

    
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True) 

    
    labels = [] 
    for i, label in enumerate(examples["ner_tags"]): 
        word_ids = tokenized_inputs.word_ids(batch_index=i) 
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token. 
        
        previous_word_idx = None 
        label_ids = []
        # Special tokens like `<s>` and `<\s>` are originally mapped to None 
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        
        for word_idx in word_ids: 
            if word_idx is None: 
                # set –100 as the label for these special tokens
                label_ids.append(-100)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token                 
                label_ids.append(label[word_idx]) 
            else: 
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100) 
                # mask the subword representations after the first subword
                 
            previous_word_idx = word_idx 
        labels.append(label_ids) 
    tokenized_inputs["labels"] = labels 
    return tokenized_inputs 

In [17]:
tokenized_datasets = conll2003.map(tokenize_and_align_labels, batched=True)

100%|██████████| 1/1 [00:00<00:00, 28.79ba/s]


In [18]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels= 69)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [19]:
from transformers import TrainingArguments, Trainer 
args = TrainingArguments( 
    "test-ner",
    evaluation_strategy = "epoch", 
    learning_rate=2e-5, 
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16, 
    num_train_epochs=5, 
    weight_decay=0.01, 
    eval_steps = 100,  
    save_total_limit = 2
) 

In [20]:
data_collator = DataCollatorForTokenClassification(tokenizer) 

In [21]:
metric = datasets.load_metric("seqeval") 

In [22]:
def compute_metrics(eval_preds): 
    pred_logits, labels = eval_preds 
    
    pred_logits = np.argmax(pred_logits, axis=2) 
    print(pred_logits)
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax
    
    predictions = []
    true_labels = []
    for i in range(len(pred_logits)):
        pred_seq = []
        true_seq = []
        for j in range(len(pred_logits[i])):
            if labels[i][j] != -100:
                pred_seq.append(pred_logits[i][j])
                true_seq.append(labels[i][j])
        predictions.append(pred_seq)
        true_labels.append(true_seq)
    
    results = metric.compute(predictions=predictions, references=true_labels) 
    return { 
   "precision": results["overall_precision"], 
   "recall": results["overall_recall"], 
   "f1": results["overall_f1"], 
  "accuracy": results["overall_accuracy"], 
  } 

In [23]:
trainer = Trainer( 
    model, 
    args, 
   train_dataset=tokenized_datasets["train"], 
   eval_dataset=tokenized_datasets["validation"], 
   data_collator=data_collator, 
   tokenizer=tokenizer, 
   compute_metrics=compute_metrics 
) 

In [24]:
trainer.train() 

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags. If tokens, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 7494
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2345
  Number of trainable parameters = 108944709
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.573670,0.841663,0.859210,0.850346,0.862779
2,1.309900,0.368309,0.906164,0.913110,0.909624,0.912908
3,0.464500,0.307247,0.918648,0.928563,0.923579,0.925774
4,0.279400,0.283649,0.927057,0.935982,0.931498,0.933881
5,0.196400,0.274555,0.930436,0.937454,0.933932,0.937117


The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags. If tokens, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3213
  Batch size = 16


[[13  5  2 ...  0  0  0]
 [13  5  2 ...  0  0  0]
 [ 5 34  6 ...  0  0  0]
 ...
 [13  9  6 ...  0  0  0]
 [13 31  6 ...  0  0  0]
 [ 9  9  2 ...  0  0  0]]


Saving model checkpoint to test-ner/checkpoint-500
Configuration saved in test-ner/checkpoint-500/config.json
Model weights saved in test-ner/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-ner/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags. If tokens, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3213
  Batch size = 16


...

[[13  5  2 ...  0  0  0]
 [13  5  2 ...  0  0  0]
 [ 5 34  6 ...  0  0  0]
 ...
 [13  9  6 ...  0  0  0]
 [13 31  6 ...  0  0  0]
 [13  9  2 ...  0  0  0]]


Saving model checkpoint to test-ner/checkpoint-2000
Configuration saved in test-ner/checkpoint-2000/config.json
Model weights saved in test-ner/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-2000/tokenizer_config.json
Special tokens file saved in test-ner/checkpoint-2000/special_tokens_map.json
Deleting older checkpoint [test-ner/checkpoint-1000] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags. If tokens, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3213
  Batch size = 16


[[13  5  2 ...  0  0  0]
 [13  5  2 ...  0  0  0]
 [ 5 34  6 ...  0  0  0]
 ...
 [13  9  6 ...  0  0  0]
 [13 31  6 ...  0  0  0]
 [13  9  2 ...  0  0  0]]




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2345, training_loss=0.5041023693613406, metrics={'train_runtime': 2578.5258, 'train_samples_per_second': 14.532, 'train_steps_per_second': 0.909, 'total_flos': 336600889275096.0, 'train_loss': 0.5041023693613406, 'epoch': 5.0})

In [25]:
model.save_pretrained("bert_all_languages")

Configuration saved in bert_all_languages/config.json
Model weights saved in bert_all_languages/pytorch_model.bin


In [26]:
model_fine_tuned_nl = AutoModelForTokenClassification.from_pretrained("bert_all_languages")

loading configuration file bert_all_languages/config.json
Model config BertConfig {
  "_name_or_path": "bert_all_languages",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL_25",
    "26": "LABEL_26",
    "27": "LABEL_27",
    "28": "LABEL_28",
  

In [27]:
from transformers import pipeline
import re
len_test = 490

In [28]:
nlp = pipeline("ner", model= model_fine_tuned_nl, tokenizer=tokenizer)

all_true_labels = []
all_prediction_labels = []
for i in range(0,len_test):
    test_data = conll2003['test'][i] 
    
    true_labels = test_data['ner_tags']
    all_true_labels.append(true_labels)
    
    tokens = test_data['tokens']
    ner_predictions = nlp(tokens)
    
    prediction_labels = []
    for i in range(0, len(ner_predictions)):
        x = ner_predictions[i]
        s = x[0]
        string = s['entity']
        label = int(re.search(r'\d+', string).group())
        prediction_labels.append(label)
    
    
    all_prediction_labels.append(prediction_labels)

Disabling tokenizer parallelism, we're using DataLoader multithreading already


In [29]:
all_prediction_labels

[[5, 2, 1, 6, 13],
 [5, 18, 21, 1, 13],
 [5, 18, 21, 22, 13],
 [5, 5, 9, 18, 21, 31, 13],
 [21, 18, 1, 3, 21, 18, 13],
 [5, 6, 19, 24, 7, 5, 9, 13],
 [9, 12, 9, 13],
 [5, 45, 39, 1, 7, 13],
 [1, 41, 9, 39, 6, 5, 13],
 [9, 6, 6, 17, 12, 3, 13],
 [18, 7, 6, 1, 5, 1, 34],
 [9, 17, 13, 1, 13],
 [5, 6, 38, 13],
 [9, 6, 21, 9, 1, 13],
 [11, 2, 5, 5, 6, 13],
 [21, 5, 39, 7, 9, 13, 13, 5, 13],
 [5, 6, 1, 13],
 [9, 6, 21, 1, 38],
 [5, 18, 21, 22, 13],
 [31, 7, 21, 17, 13],
 [5, 6, 21, 22, 13, 11, 1, 13],
 [5, 6, 5, 5, 9, 7, 13],
 [9, 11, 4, 1, 26, 18, 13],
 [13, 5, 21, 4, 39, 6, 49, 13],
 [9, 5, 26, 21, 9, 13],
 [9, 2, 19, 24, 6, 21, 14, 5, 13],
 [9, 11, 21, 12, 9, 1, 13],
 [7, 5, 5, 13],
 [9, 6, 39, 21, 4, 39, 13],
 [9, 19, 0, 5, 13],
 [9, 6, 1, 13],
 [9, 18, 1, 13],
 [9, 9, 21, 0, 11, 18, 13],
 [9, 5, 18, 21, 17, 13],
 [5, 18, 21, 1, 13],
 [5, 2, 9, 18, 13],
 [9, 9, 38, 5, 7, 13],
 [9, 18, 38, 12, 13],
 [9, 5, 5, 43, 9, 11, 11, 13],
 [9, 5, 26, 18, 24, 39, 13],
 [13, 17, 6, 1, 5, 13],
 [9, 6,

In [30]:
all_true_labels

[[17, 2, 17, 29, 13],
 [5, 18, 19, 1, 13],
 [5, 18, 21, 22, 13],
 [7, 5, 55, 18, 21, 31, 13],
 [4, 1, 18, 21, 23, 1, 13],
 [5, 2, 19, 24, 25, 21, 9, 13],
 [9, 18, 9, 13],
 [5, 20, 11, 21, 7, 13],
 [7, 5, 4, 1, 2, 16, 13],
 [9, 2, 12, 4, 1, 29, 13],
 [34, 1, 2, 21, 5, 11, 34],
 [9, 6, 21, 1, 13],
 [5, 6, 12, 13],
 [9, 2, 23, 1, 29, 13],
 [10, 2, 5, 5, 29, 13],
 [23, 16, 1, 1, 6, 21, 43, 1, 13],
 [5, 6, 1, 13],
 [9, 6, 21, 1, 38],
 [5, 18, 21, 22, 13],
 [31, 6, 21, 31, 13],
 [5, 18, 21, 22, 23, 1, 21, 13],
 [5, 2, 25, 21, 4, 7, 13],
 [9, 10, 4, 1, 26, 11, 13],
 [23, 1, 21, 4, 1, 2, 41, 13],
 [9, 6, 26, 21, 9, 13],
 [9, 2, 19, 24, 29, 21, 14, 1, 13],
 [9, 18, 23, 1, 13, 11, 13],
 [7, 5, 18, 13],
 [9, 2, 11, 21, 4, 1, 13],
 [9, 6, 0, 1, 13],
 [9, 6, 1, 13],
 [9, 18, 1, 13],
 [9, 6, 21, 0, 12, 1, 13],
 [9, 2, 29, 21, 31, 13],
 [5, 18, 23, 1, 13],
 [5, 2, 9, 29, 13],
 [13, 6, 49, 37, 7, 13],
 [9, 2, 15, 12, 13],
 [9, 2, 12, 8, 9, 45, 11, 13],
 [9, 2, 26, 19, 24, 16, 13],
 [23, 1, 2, 15, 16, 

In [31]:
results = metric.compute(predictions=all_prediction_labels, references=all_true_labels) 


In [32]:
results

{'0': {'precision': 0.6129032258064516,
  'recall': 0.3064516129032258,
  'f1': 0.4086021505376344,
  'number': 62},
 '1': {'precision': 0.32132963988919666,
  'recall': 0.3717948717948718,
  'f1': 0.3447251114413076,
  'number': 312},
 '2': {'precision': 0.5740740740740741,
  'recall': 0.2540983606557377,
  'f1': 0.35227272727272724,
  'number': 122},
 '3': {'precision': 0.8928571428571429,
  'recall': 0.76103500761035,
  'f1': 0.8216926869350862,
  'number': 657},
 '4': {'precision': 0.8586956521739131,
  'recall': 0.626984126984127,
  'f1': 0.7247706422018348,
  'number': 126},
 '5': {'precision': 0.07407407407407407,
  'recall': 0.03773584905660377,
  'f1': 0.05,
  'number': 53},
 '6': {'precision': 0.7674418604651163,
  'recall': 0.66,
  'f1': 0.7096774193548386,
  'number': 50},
 '7': {'precision': 0.18823529411764706,
  'recall': 0.4444444444444444,
  'f1': 0.2644628099173554,
  'number': 36},
 '8': {'precision': 0.3225806451612903,
  'recall': 0.4166666666666667,
  'f1': 0.3636